# Introduction to TheQ

This notebook will teach you how to run quantum circuits remotely on TheQ.

# Importing Packages

In [8]:
# Server addresses should be stable, but may occasionally be offline
import requests
import json
import math
from google.colab import files
import numpy as np
req_str = 'http://187c37408942.ngrok.io/qsim/perform_operation'
req_str_qasm = 'http://187c37408942.ngrok.io/qsim/qasm'

# Required Functions

In [3]:
# Define Create ciruit (these are the four core definitions for any circuit)
# all circuits must be bookmarked by the create_circuit and destroy_circuit command

# Initial_state is a binary integer to initalise all qubit, 
# most often initial_state=0.
def create_circuit(qubits,initial_state):
    data = {
        'operation': 'create_circuit',
        'num_qubits': qubits
    }
    result = requests.post(req_str, json=data)
    json_obj = json.loads(result.content)
    reg_id = json_obj['result']

    data = {
        'operation': 'set_state',
        'register': reg_id,
        'state': initial_state,
        'complex_value': {'re': 1, 'im': 0}
    }
    result = requests.post(req_str, json=data)
    return reg_id

# Gates.  gate_name and params are specified from the list above
def gate(register_id, gate_name, params):
    data = {
        'operation': 'gate',
        'register': register_id,
        'gate': gate_name
    }
    for k in params.keys():
        data[k] = params[k]
    result = requests.post(req_str, json=data)
    json_obj=json.loads(result.content)
    return json_obj['result']

# params contain an array of qubit numbers to measure, 'lq2m': [0,1,3] would measure qubits 0, 1 and 3 and return the 
# corresponding 3-bit integer value from 0 to 7.
def measure_qubit(register_id, params):
    data = {
        'operation': 'measure', 
        'register': register_id,
    }
    for k in params.keys():
        data[k] = params[k]
    result = requests.post(req_str, json=data)

    json_obj = json.loads(result.content)
    return json_obj['result']

# Always call destroy_circuit to terminate simulation (turn off your QC)
def destroy_circuit(register_id):
  data = {
        'operation': 'destroy_circuit', 
        'register': register_id
  }
  result = requests.post(req_str, json=data)

# Print allows you to output the current computational state of the machine.
def print_vector(register_id):
  data = {
        'operation': 'state_vector', 
        'register': register_id
  }
  result = requests.post(req_str, json=data)
  json_obj = json.loads(result.content)
  return json_obj['result']


# Creating circuits

How to simulate quantum circuits in TheQ

## Step 1: Initialize circuit

In [13]:
# Initialize the circuit
reg_id = create_circuit(2,0)

## Step 2: Apply gates

In [14]:
# X gate
gate(reg_id, 'X', {'q': 0})

True

In [15]:
# Hadamard gate
gate(reg_id, 'hadamard', {'q': 0})

True

In [16]:
# Rx gate
gate(reg_id, 'xrot', {'q': 0, 'theta': -np.pi/4})

# Ry gate
gate(reg_id, 'yrot', {'q': 0, 'theta': -np.pi/8})

# Rz gate
gate(reg_id, 'zrot', {'q': 0, 'theta': -np.pi/16})

True

In [17]:
# CNOT gate
gate(reg_id, 'cnot', {'q_control': 0, 'q_target': 1})

True

## Step 3: Measure

In [18]:
# Measure qubit 0, 1
c0 = measure_qubit(reg_id, {'lq2m': [0, 1]})
print(c0)

3


## Step 4: Destroy circuit

In [19]:
# Destroy circuit
destroy_circuit(reg_id)

# Reading the statevector

At any point in the circuit, TheQ allows one to take a sneak peak at the state.

In [23]:
# Initialize the circuit
reg_id = create_circuit(2,0)

# Hadamard gate
gate(reg_id, 'hadamard', {'q': 0})

# Hadamard gate
gate(reg_id, 'hadamard', {'q': 1})

# Print the statevector 
print(print_vector(reg_id))

# Destroy circuit
destroy_circuit(reg_id)

[{'binary_pattern': '00', 'complex_value': {'im': 0.0, 're': 0.4999999999999999}, 'state': 0}, {'binary_pattern': '01', 'complex_value': {'im': 0.0, 're': 0.4999999999999999}, 'state': 1}, {'binary_pattern': '10', 'complex_value': {'im': 0.0, 're': 0.4999999999999999}, 'state': 2}, {'binary_pattern': '11', 'complex_value': {'im': 0.0, 're': 0.4999999999999999}, 'state': 3}]
